In [163]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug  18 00:09:27 2021

@author: Testa
"""

import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import requests
import time
import numpy as np

In [703]:
import camelot
file = "/Users/christianesposito/Documents/GitHub/crunch_net/insight/foo_2.pdf"
tables = camelot.read_pdf(file)

In [706]:
import tabula
tables = tabula.read_pdf(file, pages = "all", multiple_tables = True)

Got stderr: ago 31, 2021 7:12:18 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
AVVERTENZA: New fonts found, font cache will be re-built
ago 31, 2021 7:12:18 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
AVVERTENZA: Building on-disk font cache, this may take a while
ago 31, 2021 7:12:21 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
AVVERTENZA: Finished building on-disk font cache, found 784 fonts



In [722]:
tables[10]

,Known Associates,Unnamed: 0,Unnamed: 1,Unnamed: 2
0,Name,Company,Relationships,Remarks
1,Ginevra ELKANN,Asmara Films,Non-Profit/University,Founder of Asmara Films; serves as a Director ...
2,Confirmed UHNW,NaN,Board,Committee for UNICEF with Malago.
3,Vincenzo BOCCIA,Confederazione,Non-Profit/University,President of Confindustria; serves as a Member...
4,Confirmed VHNW,Generale,Board,"Carli, where Malago serves as a Member of the ..."
5,NaN,dell'Industria Italiana,NaN,NaN
6,NaN,(Confindustria),NaN,NaN
7,Brunello CUCINELLI,Brunello Cucinelli,Non-Profit/University,Founder of Brunello Cucinelli Group; serves as...
8,Confirmed UHNW,Group,Board,Committee for UNICEF with Malago.
9,Matteo MARZOTTO,Marzotto Group,Non-Profit/University,Co-owner of the Marzotto Group; serves as a Me...


In [633]:
def clean_output(output):
    import re
    regex1 = r"(^.+?(?=Confirmed))"
    regex2 = r"(^.+?(?=Likely))"
    Family=False
    lista_connections2=[]
    lista_family2=[]
    for cell in output:
        lista=[]
        if Family==False:
            if cell==['Relation', 'Name', 'Age', 'Remarks']:
                Family=True
        if Family:
            lista.append(cell[0])
            cell[1]=cell[1].replace('\xa0','')
            matches1 = re.match(regex1, cell[1],re.MULTILINE)
            matches2 = re.match(regex2, cell[1],re.MULTILINE)
            #print(matches2.group())
            try:
                lista.append(matches1.group().strip())
#                 print('Eliminating confirmed ',matches1.group().strip())
            except:
#                 print('Except found')
                try:
                    lista.append(matches2.group().strip())
#                     print('Eliminating likely', matches2.group().strip())
                except:
#                     print('Second except found')
#                     print(cell[1].strip())
                    lista.append(cell[1].strip())
            lista.append(cell[-2])   
            lista_family2.append(lista)
        else:
            for i in range(2,5):
                lista.insert(0,cell[-i])
            cell[0]=cell[0].replace('\xa0','')
            matches1 = re.match(regex1, cell[0],re.MULTILINE)
            matches2 = re.match(regex2, cell[0],re.MULTILINE)
            try:
                lista.insert(0,matches1.group().strip())
#                 print('Eliminating confirmed ',matches1.group().strip())
            except:
#                 print('Except found')
                try:
                    lista.insert(0,matches2.group().strip())
#                     print('Eliminating likely', matches2.group().strip())
                except:
#                     print('Second except found')
#                     print(cell[0].strip())
                    lista.insert(0,cell[0].strip())
            lista_connections2.append(lista)
    #print(lista_connections2)
    return lista_connections2,lista_family2
def add_df_edges(df_edges,lista_connections,lista_family,name):
    df1=pd.DataFrame(lista_connections[1:],columns=['Name','Company','Role','Notes'])
    df2=pd.DataFrame(lista_family[1:],columns=['Relation','Name','Age'])
    df_edges=df_edges.append(df1, ignore_index=True)
    df_edges=df_edges.append(df2, ignore_index=True)
    for name_edge in df1.Name:
        df_edges.loc[df_edges.loc[:,'Name']==name_edge,'Connections']=name
    for name_edge in df2.Name:
        df_edges.loc[df_edges.loc[:,'Name']==name_edge,'Connections']=name 
    
    return df_edges

def add_df_nodes(name,output,nodes):
    list_even=['Name']
    list_uneven=[name]
    newlist=[]
    for listina in output:
        for elemento in listina:
            if elemento!='':
                newlist.append(elemento)
    for index in range(0,len(newlist),2):
        list_even.append(newlist[index])
    for index in range(1,len(newlist),2):
        list_uneven.append(newlist[index])
#     print(pd.DataFrame(list_uneven))
#     print(pd.DataFrame(list_even))
    df=pd.DataFrame([list_uneven], columns=list_even)
    nodes2=nodes.append(df, ignore_index=True)
    return nodes2

In [671]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/christianesposito/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


In [ ]:
url='https://rp.wealthx.com/secure/mylists.jsp?_c=ShUKhEo31e84c14782b89f29048d898820e2f7db5b53fc47e7b7327babae702fb4979ebea86aa57ade541fdb14f856fabd997a5e&view=2'

In [ ]:
dizionario={'nome':[]}

In [ ]:
count=0
count2=0
z = []
name_dossier=[]
company_dossier=[]
#relationships =[]
#remarks =[]
#connection =[]
net_worth_dossier=[]
#liquid_assets=[]

Prendere il nome

In [585]:
content = driver.page_source
soup = BeautifulSoup(content)
links = soup.find_all('a', attrs={'ng-click':'onClickDossier($event)'})


In [612]:
# links

In [613]:
lista_links=[]
for link in links:
    lista_links.append('https://rp.wealthx.com/secure/'+link['href'])

In [597]:
import random
random.randint(0,5)

3

In [653]:
# # lista_links
# d=-1
# for i in range(10):
#     while d!=1:
#         print('Mannaggia')
#         d=random.randint(0,5)
#         print(d)
#         continue
#     print('Fatto')

## Queries

Where should I go
Who should I meet first
Big fish there -- how do I get to it
Where should we hire (expand in the upcoming months/years)
Who should we get to in the places we already know



In [672]:
nodes

,Name,Primary Position,Source,Primary Company,Alternative Names,Estimated Net Worth,Age,Estimated Liquid Assets,Birthday,Estimated Household Wealth,Residences,Estimated Household Liquid Assets,Estimated Family's Net Worth,Hometown,Estimated Family's Liquid Assets,Religion,Wealth Trend,Marital Status
0,Alessandro MORRA,Founder,Self-Made,So.Farma.Morra,-,At least $700 million,72,At least $280 million,26 Sep 1949,-,"Naples, Campania, Italy",-,-,"Caserta, Campania, Italy",-,-,Fortune fluctuates with the performance of Co....,Married
1,Leonardo DEL VECCHIO,Chairman,Self-Made,EssilorLuxottica,-,At least $34.9 billion,86,At least $620 million,22 May 1935,At least $34.9 billion,"Milan, Lombardy, ItalyBelluno, Veneto, Italy",At least $620 million,At least $36.6 billion,"Milan, Lombardy, Italy",At least $800 million,-,Majority of fortune fluctuates with the perfor...,Married
2,Giovanni FERRERO,Chairman,Inheritance/Self-made,Ferrero International,-,At least $19.5 billion,57,At least $3.2 billion,2 Apr 1964,-,"Brussels, Brussels-Capital Region, BelgiumItal...",-,At least $29.3 billion,"Turin, Piedmont, Italy",At least $4.9 billion,-,Fortune fluctuates with the performance of Fer...,Married
3,Leonardo DEL VECCHIO,Chairman,Self-Made,EssilorLuxottica,-,At least $34.9 billion,86,At least $620 million,22 May 1935,At least $34.9 billion,"Milan, Lombardy, ItalyBelluno, Veneto, Italy",At least $620 million,At least $36.6 billion,"Milan, Lombardy, Italy",At least $800 million,-,Majority of fortune fluctuates with the perfor...,Married
4,Giovanni FERRERO,Chairman,Inheritance/Self-made,Ferrero International,-,At least $19.5 billion,57,At least $3.2 billion,2 Apr 1964,-,"ItalyBrussels, Brussels-Capital Region, Belgiu...",-,At least $29.3 billion,"Turin, Piedmont, Italy",At least $4.9 billion,-,Fortune fluctuates with the performance of Fer...,Married
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Sebastien Egon VON FURSTENBERG,Founder,Self-Made,Banca IFIS,Sebastian Egon Rainer Maria Timotheus von Fuer...,At least $530 million,71,At least $65 million,24 Jan 1950,At least $535 million,AustriaItaly,At least $65 million,-,"ItalyAustriaLausanne, Vaud, Switzerland",-,-,Fortune fluctuates with the performance of Ban...,Married
203,George Timothy CLOONEY,Co-founder,Self-Made,Smoke House Pictures,-,At least $580 million,60,At least $230 million,6 May 1961,At least $588.5 million,"New York, New York, United StatesLaglio, Lomba...",At least $231.5 million,At least $588.5 million,"Lexington, Kentucky, United States",At least $231.5 million,-,Fortune fluctuates with the value of Casamigos...,Married
204,Marco PODINI,Chairman,Inheritance/Self-made,Dedagroup,-,At least $580 million,55,At least $60 million,1966,-,"Trentino-Alto Adige, Italy",-,At least $650 million,"Bolzano, Trentino-Alto Adige, Italy",At least $130 million,-,Majority of fortune fluctuates with the perfor...,Married
205,Sergio CRAGNOTTI,Founder,Self-Made,Cragnotti & Partners Capital Investment (C&P),"L'Imperatore, Serginho",At least $580 million,81,At least $580 million,9 Jan 1940,-,Italy,-,-,"Rome, Lazio, Italy",-,-,Fortune fluctuates with the broad equity and b...,Married


In [670]:
df_edges

,Name,Company,Role,Notes,Connections,Relation,Age
0,0,0,0,0,0.0,0.0,0.0
1,Marco SALOMONI,GiVi Holding,Business - Board,Director of Givi Holding with Del Vecchio.,Marina Maria Caterina PRADA BIANCHI,NaN,NaN
2,Jean Gaston LAURENT,Covivio,Business - Colleague,"Chairman of Covivio, where Del Vecchio serves ...",Leonardo DEL VECCHIO,NaN,NaN
3,Fabrizio Riccardo DI GIUSTO,Luxottica Group,Business - Board,Former Alternate Auditor at the Luxottica Grou...,Leonardo DEL VECCHIO,NaN,NaN
4,Aldo MAZZOCCO,Generali Real Estate,Business,CEO of Generali Real Estate; served as the CEO...,Leonardo DEL VECCHIO,NaN,NaN
...,...,...,...,...,...,...,...
7595,Giuseppe LISTANTI,Confederazione Generale dell'Industria Italian...,Business - Board,Director of Confindustria with Colussi.,Angelo COLUSSI SERRAVALLO,NaN,NaN
7596,Antonio CAMPANILE,SACI,Business - Board,Director at SACI. Director of Confindustria wi...,Angelo COLUSSI SERRAVALLO,NaN,NaN
7597,Dario RINERO,Lifestyle Design,Business - Board,-,Angelo COLUSSI SERRAVALLO,NaN,NaN
7598,Giacomo COLUSSI SERRAVALLO Deceased,NaN,NaN,NaN,Angelo COLUSSI SERRAVALLO,Father,"Died in December 1999, when Colussi Serravallo..."


In [660]:
z=[1]
lista_links2=[]
for link in lista_links[204:]:
    content = driver.get(link)
    z.append(driver.current_url)
    while driver.current_url != z[-1]:
        print('mannaggia')
        continue
    try:
        content2=driver.page_source
        soup = BeautifulSoup(content2)
        tot=[]
        node_char = []
        associates = []
        names = soup.find('h1', attrs={'class':'aFEq1FsdKKIv52'})
        name=names.text.strip()
        tables = soup.find_all('table', attrs={'class':'tableBorder'})
        for table in tables:
            table_rows = table.find_all('tr')
            for tr in table_rows:
            #    td = tr.findAll('span', attrs={'class':"ng-binding"})
                td = tr.findAll()
                row = [tr.text for tr in td]
                node_char.append(row)
        tot=[]
        associates = []
        tables = soup.find_all('table', attrs={'class':'grid1 associatelink'})
        for table in tables:
            table_rows = table.find_all('tr')
            for tr in table_rows:
                td = tr.findAll('span', attrs={'class':"ng-binding"})
                td = tr.findAll()
                row = [tr.text for tr in td]
                associates.append(row)
    except:
        print('Error extracting', link)
        continue
    #links = soup.find_all('a', attrs={'ng-click':'onClickDossier($event)'})
    try:
        lista_connections,lista_family=clean_output(associates)
        df_edges=add_df_edges(df_edges,lista_connections,lista_family,name)
        nodes=add_df_nodes(name,node_char,nodes)
    except:
        print('Error ordering', name)
        continue

Error ordering Giannino  ZANICHELLI
Error ordering Roberto  ALIBARDI


KeyboardInterrupt: 

In [632]:
nodes

,Name,Primary Position,Source,Primary Company,Alternative Names,Estimated Net Worth,Age,Estimated Liquid Assets,Birthday,Estimated Household Wealth,Residences,Estimated Household Liquid Assets,Estimated Family's Net Worth,Hometown,Estimated Family's Liquid Assets,Religion,Wealth Trend,Marital Status
0,Alessandro MORRA,Founder,Self-Made,So.Farma.Morra,-,At least $700 million,72,At least $280 million,26 Sep 1949,-,"Naples, Campania, Italy",-,-,"Caserta, Campania, Italy",-,-,Fortune fluctuates with the performance of Co....,Married
1,Leonardo DEL VECCHIO,Chairman,Self-Made,EssilorLuxottica,-,At least $34.9 billion,86,At least $620 million,22 May 1935,At least $34.9 billion,"Milan, Lombardy, ItalyBelluno, Veneto, Italy",At least $620 million,At least $36.6 billion,"Milan, Lombardy, Italy",At least $800 million,-,Majority of fortune fluctuates with the perfor...,Married
2,Giovanni FERRERO,Chairman,Inheritance/Self-made,Ferrero International,-,At least $19.5 billion,57,At least $3.2 billion,2 Apr 1964,-,"Brussels, Brussels-Capital Region, BelgiumItal...",-,At least $29.3 billion,"Turin, Piedmont, Italy",At least $4.9 billion,-,Fortune fluctuates with the performance of Fer...,Married
3,Leonardo DEL VECCHIO,Chairman,Self-Made,EssilorLuxottica,-,At least $34.9 billion,86,At least $620 million,22 May 1935,At least $34.9 billion,"Milan, Lombardy, ItalyBelluno, Veneto, Italy",At least $620 million,At least $36.6 billion,"Milan, Lombardy, Italy",At least $800 million,-,Majority of fortune fluctuates with the perfor...,Married
4,Giovanni FERRERO,Chairman,Inheritance/Self-made,Ferrero International,-,At least $19.5 billion,57,At least $3.2 billion,2 Apr 1964,-,"ItalyBrussels, Brussels-Capital Region, Belgiu...",-,At least $29.3 billion,"Turin, Piedmont, Italy",At least $4.9 billion,-,Fortune fluctuates with the performance of Fer...,Married
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Alessandro MORRA,Founder,Self-Made,So.Farma.Morra,-,At least $700 million,72,At least $280 million,26 Sep 1949,-,"Naples, Campania, Italy",-,-,"Caserta, Campania, Italy",-,-,Fortune fluctuates with the performance of Co....,Married
145,Francesco ANGELINI,President,Inheritance/Self-made,Gruppo Angelini,-,At least $700 million,75,At least $55 million,8 Oct 1945,-,"Monte Carlo, Monaco, MonacoRome, Lazio, Italy",-,-,"Rome, Lazio, Italy",-,-,Fortune fluctuates with the performance of Gru...,Divorced
146,Vittorio MALACALZA,Co-owner,Self-Made,Malacalza Investimenti,-,At least $700 million,84,At least $690 million,17 Sep 1937,-,"Genoa, Liguria, Italy",-,At least $2.7 billion,"Bobbio, Emilia-Romagna, Italy",At least $730.5 million,-,Fortune fluctuates with the performance of Ban...,Married
147,Luigi BERLUSCONI,Director,Inheritance,Mediolanum,-,At least $690 million,33,At least $35 million,27 Sep 1988,-,"Macherio, Lombardy, ItalyMilan, Lombardy, Italy",-,At least $8.7 billion,"Milan, Lombardy, Italy",At least $1.8 billion,-,Fortune fluctuates with the performance of Fin...,Single


In [584]:
node_char

[['Primary Position', 'Chairman', 'Source', 'Self-Made'],
 ['Primary Company', 'EssilorLuxottica', 'Alternative Names', '-'],
 ['Estimated Net Worth', 'At least $34.9 billion', 'Age', '86'],
 ['Estimated Liquid Assets',
  'At least $620 million',
  'Birthday',
  '22 May 1935'],
 ['Estimated Household Wealth',
  'At least $34.9 billion\xa0\xa0',
  '',
  'Residences',
  'Milan, Lombardy, ItalyBelluno, Veneto, Italy',
  '',
  ''],
 ['Estimated Household Liquid Assets', 'At least $620 million\xa0\xa0', ''],
 ["Estimated Family's Net Worth",
  'At least $36.6 billion\xa0\xa0',
  '',
  'Hometown',
  'Milan, Lombardy, Italy'],
 ["Estimated Family's Liquid Assets",
  'At least $800 million\xa0\xa0',
  '',
  'Religion',
  '-'],
 ['Wealth Trend',
  'Majority of fortune fluctuates with the performance of EssilorLuxottica; majority of remainder fluctuates with the performance of Covivio and with the broad equity and bond markets.',
  'Marital Status',
  'Married']]

Prendere caratteristiche nodo

In [566]:
content = driver.page_source
soup = BeautifulSoup(content)
tot=[]
node_char = []
tables = soup.find_all('table', attrs={'class':'tableBorder'})
for table in tables:
    table_rows = table.find_all('tr')
    for tr in table_rows:
    #    td = tr.findAll('span', attrs={'class':"ng-binding"})
        td = tr.findAll()
        row = [tr.text for tr in td]
        node_char.append(row)
#pd.DataFrame(l, columns=["Name", "Company", "Net Worth", "List", "Last saved"])

Prendere gli associates e la famiglia

In [567]:
content = driver.page_source
soup = BeautifulSoup(content)
tot=[]
associates = []
tables = soup.find_all('table', attrs={'class':'grid1 associatelink'})
for table in tables:
    table_rows = table.find_all('tr')
    for tr in table_rows:
        td = tr.findAll('span', attrs={'class':"ng-binding"})
        td = tr.findAll()
        row = [tr.text for tr in td]
        associates.append(row)
#pd.DataFrame(l, columns=["Name", "Company", "Net Worth", "List", "Last saved"])

In [576]:
lista_connections,lista_family=clean_output(associates)

In [569]:
df_edges=add_df_edges(df_edges,lista_connections,lista_family,name)

In [578]:
nodes=add_df_nodes(name,node_char,nodes)

In [511]:
# To reset df_edges

In [577]:
df_edges=pd.DataFrame([[0,0,0,0,0,0,0]],columns=['Name','Company','Role','Notes','Connections','Relation','Age'])

In [574]:
nodes

,Name,Primary Position,Source,Primary Company,Alternative Names,Estimated Net Worth,Age,Estimated Liquid Assets,Birthday,Estimated Household Wealth,Residences,Estimated Household Liquid Assets,Estimated Family's Net Worth,Hometown,Estimated Family's Liquid Assets,Religion,Wealth Trend,Marital Status
0,Alessandro MORRA,Founder,Self-Made,So.Farma.Morra,-,At least $700 million,72,At least $280 million,26 Sep 1949,-,"Naples, Campania, Italy",-,-,"Caserta, Campania, Italy",-,-,Fortune fluctuates with the performance of Co....,Married
1,Leonardo DEL VECCHIO,Chairman,Self-Made,EssilorLuxottica,-,At least $34.9 billion,86,At least $620 million,22 May 1935,At least $34.9 billion,"Milan, Lombardy, ItalyBelluno, Veneto, Italy",At least $620 million,At least $36.6 billion,"Milan, Lombardy, Italy",At least $800 million,-,Majority of fortune fluctuates with the perfor...,Married
2,Giovanni FERRERO,Chairman,Inheritance/Self-made,Ferrero International,-,At least $19.5 billion,57,At least $3.2 billion,2 Apr 1964,-,"Brussels, Brussels-Capital Region, BelgiumItal...",-,At least $29.3 billion,"Turin, Piedmont, Italy",At least $4.9 billion,-,Fortune fluctuates with the performance of Fer...,Married


In [537]:
nodes

,Name,Primary Position,Source,Primary Company,Alternative Names,Estimated Net Worth,Age,Estimated Liquid Assets,Birthday,Estimated Household Wealth,Residences,Estimated Household Liquid Assets,Estimated Family's Net Worth,Hometown,Estimated Family's Liquid Assets,Religion,Wealth Trend,Marital Status
0,Alessandro MORRA,Founder,Self-Made,So.Farma.Morra,-,At least $700 million,72,At least $280 million,26 Sep 1949,-,"Naples, Campania, Italy",-,-,"Caserta, Campania, Italy",-,-,Fortune fluctuates with the performance of Co....,Married


In [ ]:
count=0
count2=0
z = []
name=[]
company=[]
relationships =[]
remarks =[]
connection =[]
net_worth=[]
liquid_assets=[]


In [ ]:
while count <= 50000:
    while count2 < 10000:
        z.append(driver.current_url)
        content = driver.page_source
        soup = BeautifulSoup(content)
        tot=[]
        for i in soup.findAll('div', attrs={'class':"longHover-module__longhover___aEanB longHover-module__longhover--hidden___36Ol9"}):
            a=i.text
            #print(a)
            tot.append(a)
        for i,j in enumerate(tot):
            if i%20==0:
                target_name.append(j)
            if i%20==1:
                stage.append(j)
            if i%20==2:
                size.append(j)
            if i%20==3:
                date.append(j)
            if i%20==4:
                investors_name.append(j)
            if i%20==5:
                url.append(j)
            if i%20==6:
                description.append(j)
            if i%20==7:
                industry.append(j)
            if i%20==8:
                target_country.append(j)
            if i%20==9:
                target_collections.append(j)
            if i%20==10:
                target_status.append(j)
            if i%20==11:
                target_ID.append(j)
            if i%20==12:
                target_founded.append(j)
            if i%20==13:
                round_simp.append(j)
            if i%20==14:
                target_continent.append(j)
            if i%20==15:
                target_state.append(j)
            if i%20==16:
                target_city.append(j)
            if i%20==17:
                target_street.append(j)
            if i%20==18:
                target_people.append(j)
            elif i%20==19:
                target_competitors.append(j)
        element = driver.find_element_by_xpath("//div[@data-test='Pagebar-Next']")
        driver.execute_script("arguments[0].click();", element)
        count2+=25
        while driver.current_url == z[-1]:
            print('mannaggia')
        else:
            continue


            #element3 = driver.find_element_by_xpath("/html/body/chrome/div/mat-sidenav-container/mat-sidenav-content/div/discover/page-layout/div/div/div[2]/section[2]/results/div/div/div[1]/div/results-info/h3/a[2]")
            #element3.click()
            #WebDriverWait(driver, 2)

    element = driver.find_element_by_xpath("(//input[@name='START_INPUT'])[3]")
    element.click()
    element.send_keys(Keys.CONTROL, 'a')
    element.send_keys(date[-1])
    count+=10000
    count2=0
    while driver.current_url == z[-1]:
        print('mannaggia')
    else:
        continue
    

In [ ]:
deals2 = pd.DataFrame({'target_name':target_name,
                      'stage':stage,
                    'size':size,
                    'date':date,
                    'investors_name':investors_name,
                    'url':url,
                    'description':description,
                    'industry':industry,
                    'target_country':target_country,
                    'target_collections':target_collections,
                    'target_status':target_status,
                    'target_ID':target_ID,
                    'target_founded':target_founded,
                    'round_simp':round_simp,
                    'target_continent':target_continent,
                    'target_state':target_state,
                    'target_city':target_city,
                     'target_street':target_street,
                     'target_people':target_people,
                     'target_competitors':target_competitors})

In [ ]:
deals2.to_csv("deals2.csv", index = False)